<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [17]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
import numpy as np
from sklearn.linear_model import LinearRegression
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [18]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next two lines for local development
#with open('../data/hopsworks-api-key.txt', 'r') as file:
    #os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158295
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "italy", "city": "emilia-romagna", "street": "gavello", "aqicn_url": "https://api.waqi.info/feed/@10009", "latitude": 11.181526893644888, "longitude": 44.925037897522905}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [19]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=2,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [20]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,20.0,italy,emilia-romagna,gavello,2024-11-13,https://api.waqi.info/feed/@10009


In [21]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style="color:#ff5f27;"> 🖍 Add the additional new features </span>

1. Mean of the pm25 for the previous three days.
2. Upwards or Downwards trend of the pm25 for the previous three days.

In [22]:
# use methods filter (to filter the last three days) and read to return a df 
last_three_days = air_quality_fg.select(['date', 'pm25']).read()
last_three_days = last_three_days.sort_values('date', ascending=False).head(3)
#print(last_three_days)

# 1. Mean PM2.5 for the Previous Three Days
# make it of type double
last_three_days['pm25'] = last_three_days['pm25'].astype(float)
# calculate the mean of the last three days
aq_today_df['pm25_3day_mean'] = last_three_days['pm25'].mean()
#print(aq_today_df)

# 2. Trend of PM2.5 for the Previous Three Days
# Function to calculate the slope of the line fitting the last 3 points
def calculate_slope(data):
    if len(data) < 3:
        return np.nan
    x = np.array([0, 1, 2]).reshape(-1, 1)
    y = data.values.reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    return model.coef_[0][0]

aq_today_df['pm25_3day_trend'] = calculate_slope(last_three_days['pm25'])

# Display the updated DataFrame with new features
aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.80s) 


,pm25,country,city,street,date,url,pm25_3day_mean,pm25_3day_trend
0,20.0,italy,emilia-romagna,gavello,2024-11-13,https://api.waqi.info/feed/@10009,51.666667,22.5


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [23]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 11.25°N 45.0°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,28.250000,0.0,16.595179,65.658920,emilia-romagna
1,2024-11-14,28.500000,0.0,16.595179,65.658920,emilia-romagna
2,2024-11-15,28.200001,0.0,19.665359,66.250504,emilia-romagna
3,2024-11-16,28.150000,0.0,14.973577,62.818974,emilia-romagna
4,2024-11-17,27.400000,0.1,18.391737,49.763721,emilia-romagna
5,2024-11-18,28.400000,0.1,23.400000,59.489841,emilia-romagna
6,2024-11-19,28.000000,0.0,22.450264,74.180748,emilia-romagna
7,2024-11-20,28.350000,0.0,17.114204,67.750931,emilia-romagna
8,2024-11-21,27.200001,0.2,12.599998,53.130020,emilia-romagna
9,2024-11-22,26.850000,1.2,11.013882,78.690102,emilia-romagna


In [24]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;"> 🖍 Add an additional new feature </span>

1. Seasonality

In [25]:
# 1. Adding Seasonality Feature
# Create a season column based on the month (e.g., Dec-Feb: Winter, Mar-May: Spring, etc.)
def assign_season(month):
    if month in [12, 1, 2]:
        #return 'winter'
        return 0
    elif month in [3, 4, 5]:
        #return 'spring'
        return 1
    elif month in [6, 7, 8]:
        #return 'summer'
        return 2
    else:
        #return 'fall'
        return 3

daily_df['season'] = daily_df['date'].dt.month.apply(assign_season)
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,season
0,2024-11-13,28.250000,0.0,16.595179,65.658920,emilia-romagna,3
1,2024-11-14,28.500000,0.0,16.595179,65.658920,emilia-romagna,3
2,2024-11-15,28.200001,0.0,19.665359,66.250504,emilia-romagna,3
3,2024-11-16,28.150000,0.0,14.973577,62.818974,emilia-romagna,3
4,2024-11-17,27.400000,0.1,18.391737,49.763721,emilia-romagna,3
5,2024-11-18,28.400000,0.1,23.400000,59.489841,emilia-romagna,3
6,2024-11-19,28.000000,0.0,22.450264,74.180748,emilia-romagna,3
7,2024-11-20,28.350000,0.0,17.114204,67.750931,emilia-romagna,3
8,2024-11-21,27.200001,0.2,12.599998,53.130020,emilia-romagna,3
9,2024-11-22,26.850000,1.2,11.013882,78.690102,emilia-romagna,3


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [26]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-13 11:48:04,567 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1345926


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1158295/jobs/named/air_quality_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fbed0fa8e50>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669724
         }
       },
       "result": {
         "observed_value": 20.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T10:48:04.000563Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "uns

In [27]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-13 11:48:32,690 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1346930


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1158295/jobs/named/weather_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fbf101e28f0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670746
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T10:48:32.000689Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
